In [2]:
!pip install ortools

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/130.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/130.2 MB 14.5 MB/s eta 0:00:09
   ---------------------------------------- 0.9/130.2 MB 9.7 MB/s eta 0:00:14
   ---------------------------------------- 1.2/130.2 MB 9.8 MB/s eta 0:00:14
    --------------------------------------- 1.9/130.2 MB 10.8 MB/s eta 0:00:12
    --------------------------------------- 2.1/130.2 MB 10.3 MB/s eta 0:00:13
    --------------------------------------- 2.4/130.2 MB 8.9 MB/s eta 0:00:15
    --------------------------------------- 2.5/130.2 MB 8.4 MB/s eta 0:00:16
    --------------------------------------- 2.7/130.2 MB 7.4 MB/s eta 0:00:18
    --------------------------------------- 2.7/130.2 MB 6.7 MB/s eta 0:00:19
    --------------------------------------- 2.8/130.2 MB 6.5 MB/s eta 0:00:20
    --------------------------------------- 2.9/130.2 MB 5.9 MB/s et

In [1]:
!pip install typing

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/78.6 kB ? eta -:--:--
     ---------------------------------------- 78.6/78.6 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26327 sha256=780569459fa9d6e8aabe3f3da2cd9a778ca8e042a932269be140b185e00715a8
  Stored in directory: c:\users\baps\appdata\local\packages\pythonsoftwarefoundation.python.3.12_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\12\98\52\2bffe242a9a487f00886e43b8ed8dac46456702e11a0d6abef
Successfully built typing


In [ ]:
"""Nurse scheduling problem with shift requests."""
from typing import Union

from ortools.sat.python import cp_model


def main() -> None:
    # This program tries to find an optimal assignment of nurses to shifts
    # (3 shifts per day, for 7 days), subject to some constraints (see below).
    # Each nurse can request to be assigned to specific shifts.
    # The optimal assignment maximizes the number of fulfilled shift requests.
    num_nurses = 5
    num_shifts = 3
    num_days = 7
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)
    shift_requests = [
        [[0, 0, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 1]],
        [[0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [0, 0, 0], [0, 0, 1]],
        [[0, 1, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 1, 0], [0, 0, 0]],
        [[0, 0, 1], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 0]],
    ]

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d, s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")

    # Each shift is assigned to exactly one nurse in .
    for d in all_days:
        for s in all_shifts:
            model.add_exactly_one(shifts[(n, d, s)] for n in all_nurses)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

    # Try to distribute the shifts evenly, so that each nurse works
    # min_shifts_per_nurse shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of nurses, some nurses will
    # be assigned one more shift.
    min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    if num_shifts * num_days % num_nurses == 0:
        max_shifts_per_nurse = min_shifts_per_nurse
    else:
        max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        num_shifts_worked: Union[cp_model.LinearExpr, int] = 0
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked += shifts[(n, d, s)]
        model.add(min_shifts_per_nurse <= num_shifts_worked)
        model.add(num_shifts_worked <= max_shifts_per_nurse)

    model.maximize(
        sum(
            shift_requests[n][d][s] * shifts[(n, d, s)]
            for n in all_nurses
            for d in all_days
            for s in all_shifts
        )
    )

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    status = solver.solve(model)

    if status == cp_model.OPTIMAL:
        print("Solution:")
        for d in all_days:
            print("Day", d)
            for n in all_nurses:
                for s in all_shifts:
                    if solver.value(shifts[(n, d, s)]) == 1:
                        if shift_requests[n][d][s] == 1:
                            print("Nurse", n, "works shift", s, "(requested).")
                        else:
                            print("Nurse", n, "works shift", s, "(not requested).")
            print()
        print(
            f"Number of shift requests met = {solver.objective_value}",
            f"(out of {num_nurses * min_shifts_per_nurse})",
        )
    else:
        print("No optimal solution found !")

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts: {solver.num_conflicts}")
    print(f"  - branches : {solver.num_branches}")
    print(f"  - wall time: {solver.wall_time}s")


if __name__ == "__main__":
    main()

In [4]:
!pip install LangChain

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip install LlamaIndex

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement LlamaIndex (from versions: none)
ERROR: No matching distribution found for LlamaIndex


In [ ]:
!pip install VectorStoreIndex
!pip install StorageContext

In [8]:
from langchain import LangChain
from llamaindex import LlamaIndex, VectorStoreIndex, StorageContext

# Sample data
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

# Initialize LlamaIndex
index = VectorStoreIndex()

# Add documents to the index
for doc in documents:
    index.add_document(doc)

# Save the index
storage_context = StorageContext()
index.save(storage_context, "my_index")

# Initialize LangChain
langchain = LangChain()

# Load the index
index = VectorStoreIndex.load(storage_context, "my_index")

# Define a query
query = "first document"

# Use LangChain to query the index
results = langchain.query(index, query)

# Print the results
for result in results:
    print(result)


ImportError: cannot import name 'LangChain' from 'langchain' (C:\Users\baps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langchain\__init__.py)